# Reproducing CB-LLMs Paper

## Setup

### Clone the [Repo](https://github.com/neil-dandekar/capstone)

In [1]:
!git clone https://github.com/neil-dandekar/capstone.git

fatal: destination path 'capstone' already exists and is not an empty directory.


In [2]:
import os
os.chdir("/content/capstone/classification")

In [3]:
!pip install -r requirements.txt

### Clone Checkpoints

In [4]:
!git lfs install
!git clone https://huggingface.co/cesun/cbllm-classification temp_repo
!mv temp_repo/mpnet_acs .
!rm -rf temp_repo

Updated git hooks.
Git LFS initialized.
Cloning into 'temp_repo'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 138 (delta 0), reused 0 (delta 0), pack-reused 135 (from 1)
Receiving objects: 100% (138/138), 20.01 KiB | 10.01 MiB/s, done.
Filtering content: 100% (117/117), 9.05 GiB | 156.52 MiB/s, done.
mv: cannot move 'temp_repo/mpnet_acs' to './mpnet_acs': Directory not empty


## Run Evaluations

### Table 2: Test Accuracy of CB-LLMs for Text Classification



The following code is aimed at replicating Table 2 of the paper.

This runs one of the evaluations from Table 2

In [5]:
# !python test_CBLLM.py --cbl_path mpnet_acs/SetFit_sst2/roberta_cbm/cbl_acc.pt

We automate this process by running all commands, collecting the outputs, and displaying them in a format like Table 2.

In [6]:
import subprocess
import json
import pandas as pd
import re

def run(cmd):
    print(f"\n=== Running: {cmd} ===\n")
    out = subprocess.check_output(cmd, shell=True, text=True)
    print(out)

    # extract accuracy from patterns like:
    # {'accuracy': 0.9406919275123559}
    match = re.search(r"\{\'accuracy\':\s*([0-9\.]+)\}", out)
    if match:
        return float(match.group(1))
    else:
        raise ValueError("Could not parse accuracy from output.")

results = []

In [7]:
BASE_CBL = "mpnet_acs"
BASE_TBM = "baseline_models/tbmc3m"
BASE_ROB = "baseline_models/roberta"

commands = [
    # CB-LLM (no ACC)
    ("CB-LLM", "SST2",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/SetFit_sst2/roberta_cbm/cbl.pt"),
    ("CB-LLM", "YelpP",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/yelp_polarity/roberta_cbm/cbl.pt"),
    ("CB-LLM", "AGnews",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/ag_news/roberta_cbm/cbl.pt"),
    ("CB-LLM", "DBpedia",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/dbpedia_14/roberta_cbm/cbl.pt"),

    # CB-LLM w/ ACC
    ("CB-LLM w/ ACC", "SST2",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/SetFit_sst2/roberta_cbm/cbl_acc.pt"),
    ("CB-LLM w/ ACC", "YelpP",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/yelp_polarity/roberta_cbm/cbl_acc.pt"),
    ("CB-LLM w/ ACC", "AGnews",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/ag_news/roberta_cbm/cbl_acc.pt"),
    ("CB-LLM w/ ACC", "DBpedia",
     f"python test_CBLLM.py --cbl_path {BASE_CBL}/dbpedia_14/roberta_cbm/cbl_acc.pt"),

    # TBM & C3M
    ("TBM&C3M", "SST2",
     f"python test_black_box.py --model_path {BASE_TBM}/backbone_finetuned_sst2.pt"),
    ("TBM&C3M", "YelpP",
     f"python test_black_box.py --model_path {BASE_TBM}/backbone_finetuned_yelp_polarity.pt"),
    ("TBM&C3M", "AGnews",
     f"python test_black_box.py --model_path {BASE_TBM}/backbone_finetuned_ag_news.pt"),
    ("TBM&C3M", "DBpedia",
     f"python test_black_box.py --model_path {BASE_TBM}/backbone_finetuned_dbpedia_14.pt"),

    # Roberta black-box baseline
    ("Roberta", "SST2",
     f"python test_black_box.py --model_path {BASE_ROB}/backbone_finetuned_sst2.pt"),
    ("Roberta", "YelpP",
     f"python test_black_box.py --model_path {BASE_ROB}/backbone_finetuned_yelp_polarity.pt"),
    ("Roberta", "AGnews",
     f"python test_black_box.py --model_path {BASE_ROB}/backbone_finetuned_ag_news.pt"),
    ("Roberta", "DBpedia",
     f"python test_black_box.py --model_path {BASE_ROB}/backbone_finetuned_dbpedia_14.pt"),
]


In [9]:
results = []

for method, dataset, cmd in commands:
    acc = run(cmd)
    results.append((method, dataset, acc))


=== Running: python test_CBLLM.py --cbl_path mpnet_acs/SetFit_sst2/roberta_cbm/cbl.pt ===

loading data...
test data len:  1821
tokenizing...
creating loader...
preparing backbone(roberta)+CBL...
get concept features...
{'accuracy': 0.9011532125205931}


=== Running: python test_CBLLM.py --cbl_path mpnet_acs/yelp_polarity/roberta_cbm/cbl.pt ===

loading data...
test data len:  38000
tokenizing...
creating loader...
preparing backbone(roberta)+CBL...
get concept features...
{'accuracy': 0.9312105263157895}


=== Running: python test_CBLLM.py --cbl_path mpnet_acs/ag_news/roberta_cbm/cbl.pt ===

loading data...
test data len:  7600
tokenizing...
creating loader...
preparing backbone(roberta)+CBL...
get concept features...
{'accuracy': 0.900921052631579}


=== Running: python test_CBLLM.py --cbl_path mpnet_acs/dbpedia_14/roberta_cbm/cbl.pt ===

loading data...
test data len:  70000
tokenizing...
creating loader...
preparing backbone(roberta)+CBL...
get concept features...
{'accuracy': 0.9

CalledProcessError: Command 'python test_black_box.py --model_path baseline_models/tbmc3m/backbone_finetuned_sst2.pt' returned non-zero exit status 1.

In [10]:
results

[('CB-LLM', 'SST2', 0.9011532125205931),
 ('CB-LLM', 'YelpP', 0.9312105263157895),
 ('CB-LLM', 'AGnews', 0.900921052631579),
 ('CB-LLM', 'DBpedia', 0.9830714285714286),
 ('CB-LLM w/ ACC', 'SST2', 0.9406919275123559),
 ('CB-LLM w/ ACC', 'YelpP', 0.9805789473684211),
 ('CB-LLM w/ ACC', 'AGnews', 0.9452631578947368),
 ('CB-LLM w/ ACC', 'DBpedia', 0.9927571428571429)]

In [11]:
df = pd.DataFrame(results, columns=["Method", "Dataset", "Accuracy"])
pivot = df.pivot(index="Method", columns="Dataset", values="Accuracy")

pivot = pivot[["SST2", "YelpP", "AGnews", "DBpedia"]]

pivot

Dataset,SST2,YelpP,AGnews,DBpedia
Method,,,,
CB-LLM,0.901153,0.931211,0.900921,0.983071
CB-LLM w/ ACC,0.940692,0.980579,0.945263,0.992757


### Table 3: Human Evaluations for Task 1 (Activation Faithfulness)

### Table 4: Human Evaluations for Task 2 (Contribution Faithfulness)


### Table 5: Accuracy, Steerability, and Perplexity of CB-LLMs for Text Generation